# 🧠 Graph-Time GCN Edge Classifier (Inductive, Snapshot-based)

## 📌 Overview

This model is designed to perform **intrusion detection** on the [CSE-CIC-IDS2018](https://www.unb.ca/cic/datasets/ids-2018.html) dataset by treating network traffic as a **dynamic graph** over time.
It learns to classify whether a network **flow (edge)** is malicious or benign, based on the structure of the network and the features of each flow.

Unlike traditional GCNs, this model is **inductive** — it **does not learn node embeddings tied to IP addresses**, meaning it can generalize to **unseen IPs** in the test set.

---

## 📊 1. Graph Construction

We represent the network traffic as a **time-evolving graph**:

* **Nodes**: IP addresses (`Src IP`, `Dst IP`)
* **Edges**: Flows between IPs (each row in the dataframe)
* **Edge features**: Traffic statistics (e.g., `Bwd Packet Length Min`, `Packet Length Std`, etc.)
* **Node features**: Computed from graph structure, not learned (e.g., degrees, activity history)
* **Labels**: Binary `target` indicating whether a flow is malicious

We **bin the data by time** (e.g., 5-minute windows). Each bin creates a snapshot graph `G_t`.

---

## 🧱 2. Feature Engineering

### 🪄 Node Features (inductive, no embedding)

For each node in a snapshot:

* `in_degree`: number of incoming flows
* `out_degree`: number of outgoing flows
* `total_degree`: sum of in/out degrees
* `prev_activity`: number of edges in the previous time bin

We use `log1p()` normalization to stabilize scale.
👉 These are **computed from structure**, so the model can handle **new IPs** without retraining.

---

### 📈 Edge Features

For each edge (flow):

* Scaled traffic statistics from the dataset (e.g., `Packet Length Std`, `SYN Flag Count`, etc.)
* **Time encoding**: sinusoidal features (sin/cos) of the timestamp to capture periodic patterns

Edge features are concatenated into a single feature vector.

---

## 🧠 3. Graph Neural Network (Message Passing)

The GNN builds **node representations** using message passing over the graph structure.

We use a stack of `GraphSAGE` (or `GCN`) layers:

[
h_v^{(l+1)} = \sigma \Big( W^{(l)} \cdot \text{AGG}\big( {h_u^{(l)} : u \in \mathcal{N}(v)} \big) \Big)
]

Where:

* ( h_v^{(l)} ) is the representation of node ( v ) at layer ( l )
* ( \mathcal{N}(v) ) are the neighbors of ( v )
* `AGG` is an aggregator (mean, sum, etc.)
* ( \sigma ) is a non-linear activation (ReLU)

✅ Because the input ( h_v^{(0)} ) are **computed features**, not learned embeddings, the model remains **inductive**.

---

## 🔍 4. Edge Classification

Once we have node embeddings ( h_{\text{src}} ) and ( h_{\text{dst}} ), we classify edges (flows) using an MLP:

[
z_{(u,v)} = [h_u , \Vert , h_v , \Vert , e_{uv}]
]

[
\hat{y}*{(u,v)} = \text{MLP}(z*{(u,v)})
]

Where:

* ( e_{uv} ) = edge features
* ( [\cdot \Vert \cdot] ) = concatenation
* ( \hat{y}_{(u,v)} ) = predicted class (benign or malicious)

This classification head outputs a 2-dimensional vector (logits) → softmax → class prediction.

---

## ⏱️ 5. Temporal Dimension

We model time as a sequence of **graph snapshots**.

* The **GNN weights are shared** across all time bins.
* Node features include **previous activity**, injecting temporal context.
* Time encoding (sin/cos) is added to edge features.

This makes the model sensitive to **temporal patterns** (e.g., bursts of malicious traffic).

---

## 🧪 6. Training & Evaluation

* **Loss**: Cross-entropy on edges (flows)
* **Optimizer**: AdamW
* **Regularization**: Dropout, weight decay
* **Metrics**: Accuracy, precision, recall, F1, confusion matrix

### 📉 Confusion Matrix

We evaluate the classifier by collecting all predictions across test snapshots:

|                    | Predicted Benign | Predicted Malicious |
| ------------------ | ---------------- | ------------------- |
| **True Benign**    | TN               | FP                  |
| **True Malicious** | FN               | TP                  |

This gives insight into false positives (FP) and false negatives (FN), which are crucial for IDS performance.

---

## 🚀 Inductive Power (Key Feature)

Unlike standard GCNs that use learned node embeddings (which break when unseen nodes appear), this model:

* **Never learns node embeddings tied to IPs**
* Computes features **directly from structure and activity**
* Generalizes to **new IP addresses in the test set**

✅ This makes it well-suited for **real-world intrusion detection**, where new devices frequently appear.

---

## 📦 Summary of the Pipeline

```mermaid
flowchart LR
    A[Raw Flows CSV] --> B[Time Binning]
    B --> C[Graph Snapshots G_t]
    C --> D[Compute Node Features]
    C --> E[Compute Edge Features + Time Encoding]
    D --> F[GNN (SAGE/GCN)]
    E --> F
    F --> G[Edge Classification Head (MLP)]
    G --> H[Prediction: Malicious / Benign]
    H --> I[Confusion Matrix + Metrics]
```

---

### ✅ Key Advantages

* **Inductive**: Handles new IPs at inference time
* **Temporal awareness**: Uses time encoding + previous activity
* **Edge-centric**: Predicts malicious flows directly
* **Graph-structured**: Exploits network structure, not just tabular features
